---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [144]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [145]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [146]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    filename = 'university_towns.txt'
    lines = []
    states = []

    with open(filename) as f:
        lines = f.readlines()

    # Remove all the useless newline characters
    lines = list(map(lambda x: x.replace('\n', ''), lines))
    edit_marker = '[edit]'
    current_state = None
    states_df = pd.DataFrame([], columns=["State", "RegionName"])

    for line in lines:
        if(line.endswith(edit_marker)):
            current_state = line.replace(edit_marker, '')
        else:
            region_split = line.split("(")
            states_df = states_df.append({
                "State": current_state,
                "RegionName": region_split[0].strip()
            }, ignore_index=True)
    
    return states_df

In [147]:
gdp_df = pd.read_excel('gdplev.xls', skiprows=5, index_col=0)

gdp_df.drop(["Unnamed: 3", "Unnamed: 7", \
             "GDP in billions of current dollars", \
             "GDP in billions of chained 2009 dollars", \
             "GDP in billions of chained 2009 dollars.1"], axis=1, inplace=True)
gdp_df.rename(columns={ "Unnamed: 4": "Quarter" }, inplace=True)
gdp_df.columns = ["Quarter", "GDP"]
# gdp_df.reset_index()

rc_df = gdp_df[gdp_df['Quarter'] > '2000'].copy()
rc_df["GDP+1"] = rc_df["GDP"].shift(-1)
rc_df["GDP+2"] = rc_df["GDP"].shift(-2)

def get_recession_start():
    # Where the GDP went down two months in a row
    recession = rc_df[(rc_df["GDP"] > rc_df["GDP+1"]) & (rc_df["GDP+1"] > rc_df["GDP+2"])]
    return recession.iloc[0].loc["Quarter"]

In [148]:
def get_recession_end():
    recession_start = get_recession_start()
    recession = rc_df[rc_df["Quarter"] > recession_start]
    
    # Where the GDP went up two months in a row
    r_start_of_end = recession[(recession["GDP"] < recession["GDP+1"]) & (recession["GDP+1"] < recession["GDP+2"])]
    return r_start_of_end.iloc[2].loc["Quarter"]

In [149]:
# MASSIVE TODO: Change to use quarter as an index

def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
     string value in a format such as 2005q3'''
    start = get_recession_start()
    end = get_recession_end()

    recession = rc_df[(rc_df["Quarter"] >= start) & (rc_df["Quarter"] <= end)]
    recession_bottom = recession[recession["GDP"] == recession["GDP"].min()]

    return recession_bottom["Quarter"].iloc[0]

In [150]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].

    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.

    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    # Name the states correctly
    df["State"] = df["State"].map(states)
    # Create a multilevel index
    df.set_index(["State", "RegionName"], inplace=True)
    # Drop un-needed columns
    df.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1, inplace=True)
    # Convert the rest of the columns to datetime
    df.columns = pd.to_datetime(df.columns)
    
    # Filter out null values
    df = df[df.notnull()]
    # Resample the columns into quarters and take the mean
    df = df[df.columns[df.columns > "1999-11"]].resample("Q", axis=1).mean()
    
    # Drop the last column (2016q4 not asked for)
    df.drop([df.columns[-1]], inplace=True, axis=1)
    # Generate quarters to rename the columns
    quarters = [str(y) + "q" + str(q) for y in range(2000, 2017) for q in range(1, 5)]
    # Rename the columns
    df.columns = quarters[:-1]
    # What the actual fuck. For some reason the values were too far to the right. I'm guessing
    # this is to do with having to drop 2016q4 but I cba to figure this out
    df = df.shift(-1, axis=1)

    return df

In [155]:
def get_ratio(row):
        return (row[r_start] - row[r_bottom]) / row[r_start]

def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    uni = get_list_of_university_towns()

    r_start  = get_recession_start()
    r_end    = get_recession_end()
    r_bottom = get_recession_bottom()

    house_df = convert_housing_data_to_quarters().copy()
    house_df = house_df.loc[:, r_start:r_end]
    house_df.reset_index(inplace=True)

    house_df["Ratio"] = house_df.apply(get_ratio, axis=1)

    uni_towns = list(get_list_of_university_towns()["RegionName"])

    house_df["uni_town"] = house_df.apply(lambda r: r["RegionName"] in uni_towns, axis=1)

    uni = house_df[house_df["uni_town"]].loc[:, "Ratio"].dropna()
    non_uni = house_df[house_df["uni_town"] == False].loc[:, "Ratio"].dropna()

    st, p = ttest_ind(non_uni, uni)

    better = "non university town" if uni.mean() > non_uni.mean() else "university town"

    return (p < 0.01, p, better)

run_ttest()

(True, 0.00036641601595526971, 'university town')